In [2]:
from vec3 import Vector3d
from ui import Simulation
import numpy as np
import matplotlib.pyplot as plt
from typing import List

# Constants
G = 6.67e-11  
AU = 14959787070

class body:
    def __init__(self,pos,vel,mass,density,color):
        self.pos= pos
        self.vel= vel
        self.mass= mass
        self.density= density
        self.color= color
        self.prev_acceleration = Vector3d(0, 0, 0)
    
    def __repr__(self):
        return (f"body(pos={self.pos}, vel={self.vel}, mass={self.mass}, "
                f"density={self.density}, color={self.color})")
    
    def force(self,other):
        
        r = other.pos - self.pos
        distance = r.mag()
        if distance == 0:
            raise ValueError("Distance cannot be zero")
        force_magnitude = G * self.mass * other.mass / distance**2
        force_vector = r.norm() * force_magnitude

        return force_vector
    
    def update(self, bodies: List['body'], dt: float):

        total_force =  Vector3d(0, 0, 0)
        for other_body in bodies:
            if self != other_body:  
                total_force += self.force(other_body)
        
        current_acceleration = total_force / self.mass
        new_position = self.pos + self.vel*dt + 0.5 * self.prev_acceleration * (dt ** 2)
        new_velocity = self.vel + 0.5 * (self.prev_acceleration + current_acceleration) * dt
        self.prev_acceleration = current_acceleration
        
        return body(new_position, new_velocity, self.mass, self.density, self.color)


def compute_energy(bodies: List[body]):
    
    kinetic_energy = 0.0
    potential_energy = 0.0
    
    for body in bodies:
        kinetic_energy += 0.5 * body.mass * body.vel.mag() ** 2

    n= len(bodies)
    for i in range(n):
        for j in range(i + 1, n):
            r = bodies[j].pos - bodies[i].pos  
            distance = r.mag()
            if distance == 0:
                continue  
            potential_energy += -G * bodies[i].mass * bodies[j].mass / distance
    
    total_energy = kinetic_energy + potential_energy
    return total_energy



def update(bodies: List[body], dt: float):
    
    updated_bodies = []
    
    for i in range(len(bodies)):
        body = bodies[i]
        total_force = Vector3d(0, 0, 0)
        
        for j in range(len(bodies)):
            if i != j:
                total_force += body.force(bodies[j])

        updated_body = body.update(total_force, dt)
        updated_bodies.append(updated_body)
    
    return updated_bodies



# Earth(planet)
planet_pos = Vector3d(0, 0, 0)  
planet_vel = Vector3d(0, 0, 0)  
planet_mass = 5.98e24  
planet_density = 1.0  
planet_color = (0, 0, 255)  
planet = body(planet_pos, planet_vel, planet_mass, planet_density, planet_color)

# Moon
moon_pos = Vector3d(3.5e8, 0, 0) 
moon_vel_mag = (G * planet_mass / moon_pos.x) ** 0.5  
moon_vel = Vector3d(0, moon_vel_mag, 0)  
moon_mass = 1.23e21  
moon_density = 1.0  
moon_color = (200, 200, 200)  
moon = body(moon_pos, moon_vel, moon_mass, moon_density, moon_color)

print("Planet:", planet)
print("\nMoon:", moon)
        
force_on_moon = planet.force(moon)
print("\nGravitational force exerted by Earth on Moon:", force_on_moon)

bodies = [planet, moon]
dt = 60 * 60
moon_updated = moon.update(bodies, dt)
print("\nUpdated Moon position and velocity after 1 hour:", moon_updated)

simulation = Simulation(
    frame_rate=30,
    width=800,
    height=800,
    caption="My Simulation",
    dt=1000,
    trail_length=500,
)
simulation.setup()
simulation.center = (0, 0)
simulation.zoom = 1e-6
simulation.loop(initial_state=bodies, update=update)







ModuleNotFoundError: No module named 'vec3'